# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [22]:
import ib_insync
print(ib_insync.__all__)
import pandas as pd

['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [3]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10, readonly=True)

<IB connected to 127.0.0.1:7497 clientId=10>

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [4]:
ib.positions()

[Position(account='U3966401', contract=Option(conId=437350001, symbol='XOM', lastTradeDateOrContractMonth='20200911', strike=38.0, right='P', multiplier='100', currency='USD', localSymbol='XOM   200911P00038000', tradingClass='XOM'), position=-1.0, avgCost=37.9133),
 Position(account='U3966401', contract=Stock(conId=270639, symbol='INTC', exchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), position=200.0, avgCost=53.045),
 Position(account='U3966401', contract=Option(conId=437256362, symbol='JPM', lastTradeDateOrContractMonth='20200911', strike=104.0, right='C', multiplier='100', currency='USD', localSymbol='JPM   200911C00104000', tradingClass='JPM'), position=-1.0, avgCost=200.2098),
 Position(account='U3966401', contract=Option(conId=437493806, symbol='GILD', lastTradeDateOrContractMonth='20200911', strike=65.0, right='P', multiplier='100', currency='USD', localSymbol='GILD  200911P00065000', tradingClass='GILD'), position=-1.0, avgCost=43.2132),
 Position(ac

Or filter the account values to get the liquidation value:

In [5]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='U3966401', tag='NetLiquidationByCurrency', value='67434.3434', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [7]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [9]:
contract = Stock('TSLA', 'SMART', 'USD')
details=ib.reqContractDetails(contract)

In [24]:
details

[ContractDetails(contract=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,AD,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,CASHQTY,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,PRICECHK,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SIZECHK,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,TPLUS1,NYSENAT,LTSE,PSX', priceMagnifier=1, underConId=0, longName='TESLA INC', contractMonth='', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST (Östliche Normalzei

### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [32]:
%time l = ib.positions()
ib.positions()

CPU times: user 15 µs, sys: 1 µs, total: 16 µs
Wall time: 21.9 µs


[Position(account='U3966401', contract=Option(conId=437350001, symbol='XOM', lastTradeDateOrContractMonth='20200911', strike=38.0, right='P', multiplier='100', currency='USD', localSymbol='XOM   200911P00038000', tradingClass='XOM'), position=-1.0, avgCost=37.9133),
 Position(account='U3966401', contract=Stock(conId=270639, symbol='INTC', exchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), position=200.0, avgCost=53.045),
 Position(account='U3966401', contract=Option(conId=437256362, symbol='JPM', lastTradeDateOrContractMonth='20200911', strike=104.0, right='C', multiplier='100', currency='USD', localSymbol='JPM   200911C00104000', tradingClass='JPM'), position=-1.0, avgCost=200.2098),
 Position(account='U3966401', contract=Option(conId=437493806, symbol='GILD', lastTradeDateOrContractMonth='20200911', strike=65.0, right='P', multiplier='100', currency='USD', localSymbol='GILD  200911P00065000', tradingClass='GILD'), position=-1.0, avgCost=43.2132),
 Position(ac

In [28]:
%time l = ib.reqPositions()

2020-09-10 09:31:00,841 ib_insync.wrapper INFO position: Position(account='U3966401', contract=Option(conId=437350001, symbol='XOM', lastTradeDateOrContractMonth='20200911', strike=38.0, right='P', multiplier='100', currency='USD', localSymbol='XOM   200911P00038000', tradingClass='XOM'), position=-1.0, avgCost=37.9133)
2020-09-10 09:31:00,842 ib_insync.wrapper INFO position: Position(account='U3966401', contract=Stock(conId=270639, symbol='INTC', exchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), position=200.0, avgCost=53.045)
2020-09-10 09:31:00,843 ib_insync.wrapper INFO position: Position(account='U3966401', contract=Option(conId=437256362, symbol='JPM', lastTradeDateOrContractMonth='20200911', strike=104.0, right='C', multiplier='100', currency='USD', localSymbol='JPM   200911C00104000', tradingClass='JPM'), position=-1.0, avgCost=200.2098)
2020-09-10 09:31:00,844 ib_insync.wrapper INFO position: Position(account='U3966401', contract=Option(conId=43749380

### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [27]:
util.logToConsole()

To see all debug messages (including network traffic):

In [11]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [33]:
ib.disconnect()

2020-09-10 09:31:48,892 ib_insync.ib INFO Disconnecting from 127.0.0.1:7497, 207 B sent in 10 messages, 65.2 kB received in 1254 messages, session time 568 s.
2020-09-10 09:31:48,894 ib_insync.client INFO Disconnecting
2020-09-10 09:31:48,899 ib_insync.client INFO Disconnected
